In [13]:
import pandas as pd
import numpy as np
from pathlib import Path

In [14]:
RAW = Path("../data/raw")
PROC = Path("../data/processed")
PROC.mkdir(parents=True, exist_ok=True)

In [15]:
from pathlib import Path
import pandas as pd

# Ruta base del proyecto (sube 3 niveles desde notebooks/data/processed/)
BASE = Path(__file__).resolve().parents[3] if "__file__" in locals() else Path().resolve().parents[2]

RAW = Path("../data/raw")
PROC = BASE / "data" / "processed"

print("Ruta base:", BASE)
print("Existe RAW?", RAW.exists())


Ruta base: /Users/sonsifabini
Existe RAW? True


In [16]:
# === 1) Cargar datos con los nombres reales ===
orders = pd.read_csv(
    RAW / "orders_202510280929.csv",
    parse_dates=["order_timestamp"],
    infer_datetime_format=True
)
clients = pd.read_csv(
    RAW / "clients_202510280926.csv",
    parse_dates=["registration_date", "last_seen"],
    infer_datetime_format=True
)
products = pd.read_csv(
    RAW / "products_202510280931.csv",
    parse_dates=["last_updated", "created_at"],
    infer_datetime_format=True
)

/var/folders/q1/tzbwph8d4qlfxl59ygs8f3dm0000gn/T/ipykernel_66228/822383569.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  orders = pd.read_csv(
/var/folders/q1/tzbwph8d4qlfxl59ygs8f3dm0000gn/T/ipykernel_66228/822383569.py:7: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  clients = pd.read_csv(
/var/folders/q1/tzbwph8d4qlfxl59ygs8f3dm0000gn/T/ipykernel_66228/822383569.py:12: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.o

In [17]:
# Tipos útiles
orders["order_price"] = orders["order_price"].astype(float)
orders["shipping_cost"] = orders["shipping_cost"].astype(float)

# === 2) Definir ventana temporal ===
# T0 como última fecha observada en órdenes válidas
# (ajustá statuses válidos según tus datos reales)
VALID_STATUS = {"delivered", "completed", "shipped"}   # ejemplo
orders_valid = orders[orders["order_status"].str.lower().isin(VALID_STATUS)].copy()

T0 = orders_valid["order_timestamp"].max().normalize()
L_days = 180
P_days = 90
L = pd.Timedelta(days=L_days)
P = pd.Timedelta(days=P_days)

obs_start  = T0 - L
pred_start = T0
pred_end   = T0 + P

# Particiones para features/label
orders_feat = orders_valid[(orders_valid["order_timestamp"] >= obs_start) &
                           (orders_valid["order_timestamp"] < T0)].copy()
orders_pred = orders_valid[(orders_valid["order_timestamp"] >= pred_start) &
                           (orders_valid["order_timestamp"] < pred_end)].copy()

# === 3) Enriquecer órdenes con info de producto (opcional)
orders_feat = orders_feat.merge(
    products[["product_id","category","brand"]],
    on="product_id", how="left"
)

In [18]:
# === 4) Construcción de features por cliente (RFM + comportamiento) ===
def build_features(orders_feat: pd.DataFrame, clients: pd.DataFrame) -> pd.DataFrame:
    base = clients[["client_id","registration_date","last_seen"]].copy()

    if orders_feat.empty:
        # Sin compras en la ventana: rellenamos defaults + recency desde last_seen
        feat = base.copy()
        feat["recency_days"]      = (T0 - feat["last_seen"]).dt.days
        feat["freq_orders"]       = 0
        feat["monetary_total"]    = 0.0
        feat["monetary_avg"]      = 0.0
        feat["monetary_max"]      = 0.0
        feat["tenure_days"]       = (T0 - feat["registration_date"]).dt.days
        feat["uniq_products"]     = 0
        feat["uniq_categories"]   = 0
        feat["uniq_brands"]       = 0
        feat["cadence_avg_days"]  = np.nan
        feat["orders_last30"]     = 0
        feat["amt_last30"]        = 0.0
        feat["pct_express"]       = 0.0
        feat["pct_standard"]      = 0.0
        feat["avg_shipping_cost"] = 0.0
        feat["pct_card"]          = 0.0
        feat["pct_cash"]          = 0.0
        feat["pct_wallet"]        = 0.0
        return feat

    of = orders_feat.copy()
    of = of.sort_values(["client_id","order_timestamp"])

    g = of.groupby("client_id", as_index=False)
    agg = g.agg(
        freq_orders     = ("order_id","count"),
        monetary_total  = ("order_price","sum"),
        monetary_avg    = ("order_price","mean"),
        monetary_max    = ("order_price","max"),
        last_order_ts   = ("order_timestamp","max"),
        first_order_ts  = ("order_timestamp","min"),
        avg_shipping_cost = ("shipping_cost","mean")
    )

    # Recency y Tenure
    agg["recency_days"] = (T0 - agg["last_order_ts"]).dt.days
    agg = agg.merge(base[["client_id","registration_date"]], on="client_id", how="left")
    agg["tenure_days"]  = (T0 - agg["registration_date"]).dt.days

    # Variedad
    uniq_products   = of.groupby("client_id")["product_id"].nunique().rename("uniq_products")
    uniq_categories = of.groupby("client_id")["category"].nunique().rename("uniq_categories")
    uniq_brands     = of.groupby("client_id")["brand"].nunique().rename("uniq_brands")
    for s in (uniq_products, uniq_categories, uniq_brands):
        agg = agg.merge(s, on="client_id", how="left")

    # Cadencia promedio
    def cadence_days(gdf):
        d = gdf["order_timestamp"].sort_values().diff().dt.days.dropna()
        return pd.Series({"cadence_avg_days": d.mean() if len(d)>0 else np.nan})
    cad = of.groupby("client_id").apply(cadence_days).reset_index()
    agg = agg.merge(cad, on="client_id", how="left")

    # Últimos 30 días
    last30 = of[of["order_timestamp"] >= (T0 - pd.Timedelta(days=30))]
    last30_agg = last30.groupby("client_id").agg(
        orders_last30=("order_id","count"),
        amt_last30=("order_price","sum")
    ).reset_index()
    agg = agg.merge(last30_agg, on="client_id", how="left").fillna({"orders_last30":0,"amt_last30":0})

    # Mix de shipping
    ship_pivot = (
        of.pivot_table(index="client_id", columns="shipping_method", values="order_id",
                       aggfunc="count", fill_value=0)
        .add_prefix("ship_").reset_index()
    )
    agg = agg.merge(ship_pivot, on="client_id", how="left")
    ship_cols = [c for c in agg.columns if c.startswith("ship_")]
    agg["ship_total"] = agg[ship_cols].sum(axis=1).replace(0, np.nan)
    agg["pct_express"]  = (agg.get("ship_express",0)  / agg["ship_total"]).fillna(0)
    agg["pct_standard"] = (agg.get("ship_standard",0) / agg["ship_total"]).fillna(0)
    agg = agg.drop(columns=ship_cols + ["ship_total"], errors="ignore")

    # Mix de payment_method (proporciones)
    pay_pivot = (
        of.pivot_table(index="client_id", columns="payment_method", values="order_id",
                       aggfunc="count", fill_value=0)
        .add_prefix("pay_").reset_index()
    )
    agg = agg.merge(pay_pivot, on="client_id", how="left")
    pay_cols = [c for c in agg.columns if c.startswith("pay_")]
    if pay_cols:
        agg["pay_total"] = agg[pay_cols].sum(axis=1).replace(0, np.nan)
        # mapeo simple a 3 buckets comunes; ajustá si tenés otros
        agg["pct_card"]   = (agg[[c for c in pay_cols if "card"   in c]].sum(axis=1) / agg["pay_total"]).fillna(0)
        agg["pct_cash"]   = (agg[[c for c in pay_cols if "cash"   in c]].sum(axis=1) / agg["pay_total"]).fillna(0)
        agg["pct_wallet"] = (agg[[c for c in pay_cols if "wallet" in c]].sum(axis=1) / agg["pay_total"]).fillna(0)
        agg = agg.drop(columns=pay_cols + ["pay_total"], errors="ignore")
    else:
        agg["pct_card"] = agg["pct_cash"] = agg["pct_wallet"] = 0.0

    # Completar con clientes sin compras en L
    feat = base.merge(agg.drop(columns=["registration_date"], errors="ignore"),
                      on="client_id", how="left")
    fill_zeros = ["freq_orders","monetary_total","monetary_avg","monetary_max",
                  "uniq_products","uniq_categories","uniq_brands",
                  "orders_last30","amt_last30","pct_express","pct_standard",
                  "avg_shipping_cost","pct_card","pct_cash","pct_wallet"]
    for c in fill_zeros:
        feat[c] = feat[c].fillna(0 if c!="avg_shipping_cost" else 0.0)
    feat["recency_days"] = feat["recency_days"].fillna((T0 - feat["last_seen"]).dt.days)
    feat["tenure_days"]  = feat["tenure_days"].fillna((T0 - feat["registration_date"]).dt.days)
    return feat.drop(columns=["last_seen"])

features = build_features(orders_feat, clients)

/var/folders/q1/tzbwph8d4qlfxl59ygs8f3dm0000gn/T/ipykernel_66228/1848591849.py:58: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cad = of.groupby("client_id").apply(cadence_days).reset_index()
